In [4]:
!pip install mediapipe

import cv2
import mediapipe as mp
import mediapipe.solutions as mp_solutions
import numpy as np
import math

VIDEO_PATH = '/kaggle/input/newtoetap/Toe Taps.mp4'
OUTPUT_VIDEO_PATH = 'annotated_toe_taps_final.mp4'

TOUCH_THRESHOLD = 35
TOUCH_COOLDOWN_FRAMES = 8
VELOCITY_FRAME_DELTA = 5

LOWER_COLOR_BOUND = np.array([0, 0, 150])
UPPER_COLOR_BOUND = np.array([180, 60, 255])

mp_pose = mp_solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.6)

def find_ball_optimized(frame):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_frame, LOWER_COLOR_BOUND, UPPER_COLOR_BOUND)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    best_candidate = None
    best_score = 0
    if len(contours) > 0:
        for c in contours:
            area = cv2.contourArea(c)
            if area < 100 or area > 5000:
                continue
            perimeter = cv2.arcLength(c, True)
            if perimeter == 0:
                continue
            circularity = 4 * np.pi * (area / (perimeter * perimeter))
            if 0.7 < circularity < 1.2:
                if circularity > best_score:
                    best_score = circularity
                    best_candidate = c
    if best_candidate is not None:
        ((x, y), radius) = cv2.minEnclosingCircle(best_candidate)
        M = cv2.moments(best_candidate)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        bbox = (int(x - radius), int(y - radius), int(radius * 2), int(radius * 2))
        return center, bbox
    return None, None

def run_video_analysis(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file at '{video_path}'. Please check the path.")
        return
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) if int(cap.get(cv2.CAP_PROP_FPS)) > 0 else 30
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (width, height))
    print("Starting video processing...")
    touch_count_left, touch_count_right = 0, 0
    ball_rotation, player_velocity = "N/A", 0.0
    touch_cooldown = 0
    player_positions, prev_gray_frame = [], None
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if prev_gray_frame is None:
            prev_gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ball_center, ball_bbox = find_ball_optimized(frame)
        if ball_center:
            cv2.rectangle(frame, (ball_bbox[0], ball_bbox[1]), 
                          (ball_bbox[0] + ball_bbox[2], ball_bbox[1] + ball_bbox[3]),
                          (0, 255, 0), 2)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            left_toe = (int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x * width), int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y * height))
            right_toe = (int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].x * width), int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y * height))
            left_hip = (int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * width), int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * height))
            right_hip = (int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x * width), int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y * height))
            player_center = ((left_hip[0] + right_hip[0]) / 2, (left_hip[1] + right_hip[1]) / 2)
            player_positions.append(player_center)
            if len(player_positions) > VELOCITY_FRAME_DELTA:
                player_positions.pop(0)
            if ball_center and touch_cooldown == 0:
                dist_left = math.hypot(ball_center[0] - left_toe[0], ball_center[1] - left_toe[1])
                dist_right = math.hypot(ball_center[0] - right_toe[0], ball_center[1] - right_toe[1])
                if min(dist_left, dist_right) < TOUCH_THRESHOLD:
                    touch_cooldown = TOUCH_COOLDOWN_FRAMES
                    if len(player_positions) == VELOCITY_FRAME_DELTA:
                        past_pos = player_positions[0]
                        pixel_dist = math.hypot(player_center[0] - past_pos[0], player_center[1] - past_pos[1])
                        time_elapsed = VELOCITY_FRAME_DELTA / fps
                        player_velocity = pixel_dist / time_elapsed if time_elapsed > 0 else 0
                    if dist_left < dist_right:
                        touch_count_left += 1
                    else:
                        touch_count_right += 1
        if ball_bbox:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            ball_roi = gray_frame[ball_bbox[1]:ball_bbox[1]+ball_bbox[3], ball_bbox[0]:ball_bbox[0]+ball_bbox[2]]
            prev_ball_roi = prev_gray_frame[ball_bbox[1]:ball_bbox[1]+ball_bbox[3], ball_bbox[0]:ball_bbox[0]+ball_bbox[2]]
            if ball_roi.size > 0 and prev_ball_roi.size > 0 and ball_roi.shape == prev_ball_roi.shape:
                flow = cv2.calcOpticalFlowFarneback(prev_ball_roi, ball_roi, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                avg_flow_x = np.mean(flow[..., 0])
                if avg_flow_x > 1.0: ball_rotation = "Forward"
                elif avg_flow_x < -1.0: ball_rotation = "Backward"
            prev_gray_frame = gray_frame
        if touch_cooldown > 0:
            touch_cooldown -= 1
        cv2.rectangle(frame, (0, height-100), (width, height), (0, 0, 0), -1)
        cv2.putText(frame, f"Right Leg Taps: {touch_count_right}", (20, height - 65), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Left Leg Taps: {touch_count_left}", (20, height - 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Rotation: {ball_rotation}", (width - 450, height - 65), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Velocity: {player_velocity:.2f} px/sec", (width - 450, height - 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        out.write(frame)
    cap.release()
    out.release()
    print(f"✅ Processing complete! Annotated video saved to '{OUTPUT_VIDEO_PATH}'")

run_video_analysis(VIDEO_PATH)


ModuleNotFoundError: No module named 'mediapipe.solutions'

In [3]:
# A reliable way to test internet connectivity in any environment
!curl -I https://www.google.com

HTTP/2 200 
content-type: text/html; charset=ISO-8859-1
content-security-policy-report-only: object-src 'none';base-uri 'self';script-src 'nonce-17UoX7lKC8QRAhP9HXOWOg' 'strict-dynamic' 'report-sample' 'unsafe-eval' 'unsafe-inline' https: http:;report-uri https://csp.withgoogle.com/csp/gws/other-hp
accept-ch: Sec-CH-Prefers-Color-Scheme
p3p: CP="This is not a P3P policy! See g.co/p3phelp for more info."
date: Wed, 20 Aug 2025 18:28:47 GMT
server: gws
x-xss-protection: 0
x-frame-options: SAMEORIGIN
expires: Wed, 20 Aug 2025 18:28:47 GMT
cache-control: private
set-cookie: AEC=AVh_V2jy34USzA22ppB5n-xqLFaziPk8Qmtlxg9P9yDMuyqBTn4dSjhZBg; expires=Mon, 16-Feb-2026 18:28:47 GMT; path=/; domain=.google.com; Secure; HttpOnly; SameSite=lax
set-cookie: __Secure-ENID=28.SE=koOB34egDrEbCv2klbHI8d03wvQe2mtG2s3-y9_l9iUmHhG517Sh8fxxpNdqW2LEyY2JAK3hMoGIkBogY8hvxIWDp8Ol0QcHdqMcI0EwavUUf2pYUuWNTMxfqg-RlTf3BHb6rLlOhpNt5spMiwgJfvtm-0F7kXEN5sGfK0O2ASlp7GZSbH51y5ZRsAureX_keBS8GPPvh5Ldo6B8CctomJU8_MAwnij-2Pdlw